# DAT535 - Final Project: Main Pipeline
## Notebook 3: Part 3 - Gold Layer Analysis

**Purpose:** To take the clean, conformed data from the Silver Layer and create aggregated, business-level insights for our use case.

**Use Case:** "Analyze 4 years (2020-2024) of NYC Taxi data to identify key operational trends and passenger behaviors for a business intelligence dashboard."

Each analysis below represents a pre-computed table ready for visualization (a "Gold" table).

## 1. Setup: Imports and Spark Session

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, hour, avg, count, sum, when, desc


spark = SparkSession.builder \
    .appName("TLC Gold Layer Analysis") \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("Spark Session initialized for Gold Layer Analysis.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/17 19:59:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session initialized for Gold Layer Analysis.


## 2. Load the Silver Layer Data

We begin by loading the clean, partitioned Parquet data that was the output of our Part 2 pipeline.

In [2]:
SILVER_PATH = "/home/ubuntu/project/silver_layer_data_v2"
GOLD_BASE_PATH = "/home/ubuntu/project/gold_layer_data"

print(f"Loading Silver Layer data from: {SILVER_PATH}")

# Spark will automatically discover the partitions (e.g., pickup_year=2020, taxi_type=yellow)
silver_df = spark.read.parquet(SILVER_PATH)

# Filter out records with parse errors for clean Gold layer analytics
# Keep only high-quality records without missing values
silver_df_clean = silver_df.filter(
    (col("has_parse_errors") == False) & 
    (col("has_missing_values") == False)
)

# Cache the DataFrame for faster iterative queries
silver_df_clean.cache()

# Perform a quick verification to ensure the data is loaded correctly.
print("\nSilver Layer data loaded successfully.")
print(f"Total records in Silver Layer: {silver_df.count():,}")
print(f"Clean records (no parse/missing errors): {silver_df_clean.count():,}")
print("\nSchema of the Silver DataFrame:")
silver_df_clean.printSchema()
print("\nBreakdown by taxi_type:")
silver_df_clean.groupBy("taxi_type").count().show()
print("\nBreakdown by year:")
silver_df_clean.groupBy("pickup_year").count().orderBy("pickup_year").show()

Loading Silver Layer data from: /home/ubuntu/project/silver_layer_data_v2



Silver Layer data loaded successfully.


Total records in Silver Layer: 179,779,179


25/11/17 20:00:26 WARN MemoryStore: Not enough space to cache rdd_13_31 in memory! (computed 71.5 MiB so far)
25/11/17 20:00:27 WARN BlockManager: Persisting block rdd_13_31 to disk instead.


25/11/17 20:00:27 WARN MemoryStore: Not enough space to cache rdd_13_25 in memory! (computed 122.7 MiB so far)
25/11/17 20:00:27 WARN BlockManager: Persisting block rdd_13_25 to disk instead.


...

25/11/17 20:02:01 WARN MemoryStore: Not enough space to cache rdd_13_93 in memory! (computed 69.7 MiB so far)
25/11/17 20:02:01 WARN MemoryStore: Not enough space to cache rdd_13_95 in memory! (computed 69.7 MiB so far)


25/11/17 20:02:01 WARN MemoryStore: Not enough space to cache rdd_13_97 in memory! (computed 68.0 MiB so far)
25/11/17 20:02:01 WARN MemoryStore: Not enough space to cache rdd_13_100 in memory! (computed 34.8 MiB so far)


25/11/17 20:02:02 WARN MemoryStore: Not enough space to cache rdd_13_103 in memory! (computed 68.8 MiB so far)
25/11/17 20:02:02 WARN MemoryStore: Not enough space to cache rdd_13_101 in memory! (computed 120.4 MiB so far)


25/11/17 20:02:02 WARN MemoryStore: Not enough space to cache rdd_13_102 in memory! (computed 186.8 MiB so far)
25/11/17 20:02:02 WARN MemoryStore: Not enough space to cache rdd_13_106 in memory! (computed 121.0 MiB so far)


+-----------+--------+
|pickup_year|   count|
+-----------+--------+
|       2020|25045207|
|       2021|30081583|
|       2022|39037373|
|       2023|37732224|
|       2024|37714308|
+-----------+--------+



## 3. Gold Analysis 1: Hourly Demand Patterns

**Business Question:** What are the busiest hours of the day for taxi pickups? How does this pattern differ between Yellow and Green taxis?

In [3]:
print("--- Generating Gold Table 1: Hourly Demand Patterns ---")

# Use the existing pickup_hour column from Silver layer
# No need to derive it again - it's already in the schema
gold_hourly_demand = silver_df_clean.groupBy("taxi_type", "pickup_hour") \
    .agg(
        count("*").alias("total_trips"),
        avg("total_amount").alias("average_fare"),
        avg("trip_distance").alias("average_distance_miles")
    ) \
    .orderBy("taxi_type", "pickup_hour")

print("\n--- Gold Table 1: Hourly Demand Patterns ---")
gold_hourly_demand.show(48) # Show all 48 rows (24 hours * 2 taxi types)

# Save the Gold table to Parquet for future use by a dashboard.
print(f"Saving hourly_demand Gold table to: {GOLD_BASE_PATH}/hourly_demand")
gold_hourly_demand.write.mode("overwrite").parquet(f"{GOLD_BASE_PATH}/hourly_demand")
print("✓ Saved successfully.")

--- Generating Gold Table 1: Hourly Demand Patterns ---

--- Gold Table 1: Hourly Demand Patterns ---


25/11/17 20:02:03 WARN MemoryStore: Not enough space to cache rdd_13_0 in memory! (computed 70.4 MiB so far)


25/11/17 20:02:03 WARN MemoryStore: Not enough space to cache rdd_13_6 in memory! (computed 69.8 MiB so far)
25/11/17 20:02:03 WARN MemoryStore: Not enough space to cache rdd_13_2 in memory! (computed 71.0 MiB so far)
25/11/17 20:02:03 WARN MemoryStore: Not enough space to cache rdd_13_5 in memory! (computed 71.5 MiB so far)
25/11/17 20:02:03 WARN MemoryStore: Not enough space to cache rdd_13_1 in memory! (computed 70.8 MiB so far)
25/11/17 20:02:03 WARN MemoryStore: Not enough space to cache rdd_13_7 in memory! (computed 68.8 MiB so far)


25/11/17 20:02:04 WARN MemoryStore: Not enough space to cache rdd_13_9 in memory! (computed 123.3 MiB so far)
25/11/17 20:02:04 WARN MemoryStore: Not enough space to cache rdd_13_11 in memory! (computed 71.5 MiB so far)
25/11/17 20:02:04 WARN MemoryStore: Not enough space to cache rdd_13_10 in memory! (computed 123.0 MiB so far)
25/11/17 20:02:04 WARN MemoryStore: Not enough space to cache rdd_13_13 in memory! (computed 121.4 MiB so far)


25/11/17 20:02:04 WARN MemoryStore: Not enough space to cache rdd_13_15 in memory! (computed 187.6 MiB so far)


...

25/11/17 20:02:22 WARN MemoryStore: Not enough space to cache rdd_13_92 in memory! (computed 70.3 MiB so far)
25/11/17 20:02:22 WARN MemoryStore: Not enough space to cache rdd_13_94 in memory! (computed 18.2 MiB so far)
25/11/17 20:02:22 WARN MemoryStore: Not enough space to cache rdd_13_95 in memory! (computed 35.7 MiB so far)
25/11/17 20:02:22 WARN MemoryStore: Not enough space to cache rdd_13_93 in memory! (computed 69.7 MiB so far)
25/11/17 20:02:22 WARN MemoryStore: Not enough space to cache rdd_13_96 in memory! (computed 35.2 MiB so far)


25/11/17 20:02:23 WARN MemoryStore: Not enough space to cache rdd_13_97 in memory! (computed 118.2 MiB so far)


25/11/17 20:02:23 WARN MemoryStore: Not enough space to cache rdd_13_102 in memory! (computed 18.4 MiB so far)
25/11/17 20:02:23 WARN MemoryStore: Not enough space to cache rdd_13_103 in memory! (computed 18.2 MiB so far)
25/11/17 20:02:23 WARN MemoryStore: Not enough space to cache rdd_13_101 in memory! (computed 70.2 MiB so far)
25/11/17 20:02:23 WARN MemoryStore: Not enough space to cache rdd_13_100 in memory! (computed 68.2 MiB so far)


25/11/17 20:02:24 WARN MemoryStore: Not enough space to cache rdd_13_107 in memory! (computed 34.7 MiB so far)


✓ Saved successfully.


## 4. Gold Analysis 2: Payment Type Insights

**Business Question:** Which payment types are most common and which generate the most revenue? How does tipping behavior change with payment type?

In [4]:
print("\n--- Generating Gold Table 2: Payment Type Insights ---")

# We'll create a more readable payment type name using a WHEN clause.
# Based on the TLC data dictionary: 1=Credit card, 2=Cash, 3=No charge, 4=Dispute
df_with_payment_name = silver_df_clean.withColumn("payment_name",
    when(col("payment_type") == 1, "Credit Card")
    .when(col("payment_type") == 2, "Cash")
    .when(col("payment_type") == 3, "No Charge")
    .when(col("payment_type") == 4, "Dispute")
    .otherwise("Unknown")
)

# Group by the readable payment name to create the aggregation.
gold_payment_analysis = df_with_payment_name.groupBy("payment_name") \
    .agg(
        count("*").alias("total_trips"),
        sum("total_amount").alias("total_revenue"),
        avg("fare_amount").alias("average_fare"),
        avg("tip_amount").alias("average_tip")
    ) \
    .orderBy(desc("total_trips"))

print("\n--- Gold Table 2: Payment Type Analysis ---")
gold_payment_analysis.show()

# Save the Gold table.
print(f"Saving payment_analysis Gold table to: {GOLD_BASE_PATH}/payment_analysis")
gold_payment_analysis.write.mode("overwrite").parquet(f"{GOLD_BASE_PATH}/payment_analysis")
print("✓ Saved successfully.")


--- Generating Gold Table 2: Payment Type Insights ---

--- Gold Table 2: Payment Type Analysis ---


25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_7 in memory! (computed 34.6 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_4 in memory! (computed 36.0 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_0 in memory! (computed 36.1 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_1 in memory! (computed 35.7 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_2 in memory! (computed 35.9 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_3 in memory! (computed 36.2 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_5 in memory! (computed 36.3 MiB so far)
25/11/17 20:02:25 WARN MemoryStore: Not enough space to cache rdd_13_6 in memory! (computed 35.7 MiB so far)


25/11/17 20:02:26 WARN MemoryStore: Not enough space to cache rdd_13_8 in memory! (computed 71.5 MiB so far)
25/11/17 20:02:26 WARN MemoryStore: Not enough space to cache rdd_13_10 in memory! (computed 71.4 MiB so far)
25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_14 in memory! (computed 69.6 MiB so far)


25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_13 in memory! (computed 121.4 MiB so far)
25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_15 in memory! (computed 121.6 MiB so far)
25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_11 in memory! (computed 189.6 MiB so far)


25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_19 in memory! (computed 71.0 MiB so far)
25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_23 in memory! (computed 36.0 MiB so far)
25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_16 in memory! (computed 120.4 MiB so far)
25/11/17 20:02:27 WARN MemoryStore: Not enough space to cache rdd_13_22 in memory! (computed 119.7 MiB so far)


...

25/11/17 20:02:43 WARN MemoryStore: Not enough space to cache rdd_13_95 in memory! (computed 18.6 MiB so far)
25/11/17 20:02:43 WARN MemoryStore: Not enough space to cache rdd_13_90 in memory! (computed 67.5 MiB so far)
25/11/17 20:02:43 WARN MemoryStore: Not enough space to cache rdd_13_94 in memory! (computed 68.8 MiB so far)
25/11/17 20:02:43 WARN MemoryStore: Not enough space to cache rdd_13_91 in memory! (computed 119.6 MiB so far)


25/11/17 20:02:44 WARN MemoryStore: Not enough space to cache rdd_13_99 in memory! (computed 35.8 MiB so far)
25/11/17 20:02:44 WARN MemoryStore: Not enough space to cache rdd_13_102 in memory! (computed 18.4 MiB so far)


25/11/17 20:02:44 WARN MemoryStore: Not enough space to cache rdd_13_103 in memory! (computed 18.2 MiB so far)
25/11/17 20:02:44 WARN MemoryStore: Not enough space to cache rdd_13_101 in memory! (computed 70.2 MiB so far)


25/11/17 20:02:44 WARN MemoryStore: Not enough space to cache rdd_13_104 in memory! (computed 67.5 MiB so far)
25/11/17 20:02:44 WARN MemoryStore: Not enough space to cache rdd_13_107 in memory! (computed 34.7 MiB so far)


✓ Saved successfully.


## 5. Gold Analysis 3: Trip Distance Distribution

**Business Question:** What are the most common trip distances? Are there differences in how Yellow and Green taxis are used for short vs. long trips?

In [5]:
print("\n--- Generating Gold Table 3: Trip Distance Distribution ---")

# Create "distance buckets" to categorize each trip.
df_with_buckets = silver_df_clean.withColumn("distance_bucket",
    when(col("trip_distance") <= 1, "A: 0-1 Miles (Short Hop)")
    .when((col("trip_distance") > 1) & (col("trip_distance") <= 3), "B: 1-3 Miles (Standard)")
    .when((col("trip_distance") > 3) & (col("trip_distance") <= 5), "C: 3-5 Miles (Medium)")
    .when((col("trip_distance") > 5) & (col("trip_distance") <= 10), "D: 5-10 Miles (Long)")
    .otherwise("E: 10+ Miles (Very Long)")
)

# Group by taxi_type and the new distance buckets.
gold_distance_analysis = df_with_buckets.groupBy("taxi_type", "distance_bucket") \
    .agg(
        count("*").alias("total_trips"),
        avg("total_amount").alias("average_fare")
    ) \
    .orderBy("taxi_type", "distance_bucket")

print("\n--- Gold Table 3: Trip Distance Analysis ---")
gold_distance_analysis.show()

# Save the Gold table.
print(f"Saving distance_analysis Gold table to: {GOLD_BASE_PATH}/distance_analysis")
gold_distance_analysis.write.mode("overwrite").parquet(f"{GOLD_BASE_PATH}/distance_analysis")
print("✓ Saved successfully.")


--- Generating Gold Table 3: Trip Distance Distribution ---

--- Gold Table 3: Trip Distance Analysis ---


25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_3 in memory! (computed 36.2 MiB so far)
25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_6 in memory! (computed 35.7 MiB so far)
25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_1 in memory! (computed 35.7 MiB so far)
25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_4 in memory! (computed 36.0 MiB so far)
25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_2 in memory! (computed 35.9 MiB so far)
25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_7 in memory! (computed 34.6 MiB so far)
25/11/17 20:02:45 WARN MemoryStore: Not enough space to cache rdd_13_0 in memory! (computed 70.4 MiB so far)


25/11/17 20:02:46 WARN MemoryStore: Not enough space to cache rdd_13_12 in memory! (computed 35.8 MiB so far)
25/11/17 20:02:47 WARN MemoryStore: Not enough space to cache rdd_13_13 in memory! (computed 36.3 MiB so far)


25/11/17 20:02:47 WARN MemoryStore: Not enough space to cache rdd_13_11 in memory! (computed 123.1 MiB so far)
25/11/17 20:02:47 WARN MemoryStore: Not enough space to cache rdd_13_10 in memory! (computed 123.0 MiB so far)
25/11/17 20:02:47 WARN MemoryStore: Not enough space to cache rdd_13_14 in memory! (computed 120.9 MiB so far)


25/11/17 20:02:47 WARN MemoryStore: Not enough space to cache rdd_13_15 in memory! (computed 121.6 MiB so far)


...

25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_96 in memory! (computed 35.2 MiB so far)
25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_98 in memory! (computed 17.7 MiB so far)
25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_94 in memory! (computed 68.8 MiB so far)
25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_95 in memory! (computed 69.7 MiB so far)


25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_101 in memory! (computed 36.3 MiB so far)
25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_103 in memory! (computed 18.2 MiB so far)


25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_105 in memory! (computed 68.2 MiB so far)
25/11/17 20:03:06 WARN MemoryStore: Not enough space to cache rdd_13_104 in memory! (computed 34.4 MiB so far)


25/11/17 20:03:07 WARN MemoryStore: Not enough space to cache rdd_13_102 in memory! (computed 118.7 MiB so far)
25/11/17 20:03:07 WARN MemoryStore: Not enough space to cache rdd_13_106 in memory! (computed 121.0 MiB so far)


✓ Saved successfully.


## 6. Cleanup

Stop the Spark session to release cluster resources.

In [6]:
spark.stop()
print("\nGold Layer processing complete. Spark session stopped.")


Gold Layer processing complete. Spark session stopped.
